In [3]:

# Training on Nvidia 4070Ti super 16gb VRAM.
# Import necessary libraries
import functools
import gc
import itertools
import json
import logging
import math
import os
import random
import shutil
from contextlib import nullcontext
from pathlib import Path
from typing import List, Union

import accelerate
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
import torchvision.transforms.functional as TF
import transformers
import webdataset as wds
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration, set_seed
from braceexpand import braceexpand
from packaging import version
from peft import LoraConfig, get_peft_model, get_peft_model_state_dict
from torch.utils.data import default_collate
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import AutoTokenizer, CLIPTextModel, PretrainedConfig

import diffusers
from diffusers import (
    AutoencoderKL,
    DDPMScheduler,
    LCMScheduler,
    StableDiffusionPipeline,
    UNet2DConditionModel,
)
from diffusers.optimization import get_scheduler
from diffusers.training_utils import resolve_interpolation_mode
from diffusers.utils import check_min_version, is_wandb_available
from diffusers.utils.import_utils import is_xformers_available
from webdataset.tariterators import (
    base_plus_ext,
    tar_file_expander,
    url_opener,
    valid_sample,
)


MAX_SEQ_LENGTH = 77

# Will error if the minimal version of diffusers is not installed. Remove at your own risks.
check_min_version("0.29.0.dev0")

logger = get_logger(__name__)

In [4]:
# Configuration
config = {
    "pretrained_teacher_model": "runwayml/stable-diffusion-v1-5",
    "output_dir": "/home/node/academic/lcm/outputs",
    "mixed_precision": "fp16",
    "resolution": 512,
    "lora_rank": 128,
    "learning_rate": 1e-4,
    "loss_type": "huber",
    "adam_weight_decay": 0.0,
    "max_train_steps": 1000,
    "max_train_samples": 130000,
    "dataloader_num_workers": 8,
    "train_shards_path_or_url": "/home/node/academic/lcmlora/00000.tar",
    "validation_steps": 200,
    "checkpointing_steps": 200,
    "checkpoints_total_limit": 10,
    "train_batch_size": 1,
    "gradient_accumulation_steps": ,
    "use_8bit_adam": True,
    "seed": 453645634,
    "lr_scheduler": "constant",  
    "lr_warmup_steps": 500,  
    "num_train_epochs": 100,  
    "revision": None,  
    "vae_encode_batch_size": 32,  
    "w_min": 5.0,  
    "w_max": 15.0,  
    "enable_xformers_memory_efficient_attention": False,  
}

In [5]:
def get_module_kohya_state_dict(module, prefix: str, dtype: torch.dtype, adapter_name: str = "default"):
    kohya_ss_state_dict = {}
    for peft_key, weight in get_peft_model_state_dict(module, adapter_name=adapter_name).items():
        kohya_key = peft_key.replace("base_model.model", prefix)
        kohya_key = kohya_key.replace("lora_A", "lora_down")
        kohya_key = kohya_key.replace("lora_B", "lora_up")
        kohya_key = kohya_key.replace(".", "_", kohya_key.count(".") - 2)
        kohya_ss_state_dict[kohya_key] = weight.to(dtype)

        # Set alpha parameter
        if "lora_down" in kohya_key:
            alpha_key = f'{kohya_key.split(".")[0]}.alpha'
            kohya_ss_state_dict[alpha_key] = torch.tensor(module.peft_config[adapter_name].lora_alpha).to(dtype)

    return kohya_ss_state_dict


def filter_keys(key_set):
    def _f(dictionary):
        return {k: v for k, v in dictionary.items() if k in key_set}

    return _f


def group_by_keys_nothrow(data, keys=base_plus_ext, lcase=True, suffixes=None, handler=None):
    current_sample = None
    for filesample in data:
        assert isinstance(filesample, dict)
        fname, value = filesample["fname"], filesample["data"]
        prefix, suffix = keys(fname)
        if prefix is None:
            continue
        if lcase:
            suffix = suffix.lower()
        if current_sample is None or prefix != current_sample["__key__"] or suffix in current_sample:
            if valid_sample(current_sample):
                yield current_sample
            current_sample = {"__key__": prefix, "__url__": filesample["__url__"]}
        if suffixes is None or suffix in suffixes:
            current_sample[suffix] = value
    if valid_sample(current_sample):
        yield current_sample


def tarfile_to_samples_nothrow(src, handler=wds.warn_and_continue):
    streams = url_opener(src, handler=handler)
    files = tar_file_expander(streams, handler=handler)
    samples = group_by_keys_nothrow(files, handler=handler)
    return samples


class WebdatasetFilter:
    def __init__(self, min_size=1024, max_pwatermark=0.5):
        self.min_size = min_size
        self.max_pwatermark = max_pwatermark

    def __call__(self, x):
        try:
            if "json" in x:
                x_json = json.loads(x["json"])
                filter_size = (x_json.get("original_width", 0.0) or 0.0) >= self.min_size and x_json.get(
                    "original_height", 0
                ) >= self.min_size
                filter_watermark = (x_json.get("pwatermark", 1.0) or 1.0) <= self.max_pwatermark
                return filter_size and filter_watermark
            else:
                return False
        except Exception:
            return False

In [6]:
# Load scheduler, tokenizer, text encoder, VAE and teacher U-Net
noise_scheduler = DDPMScheduler.from_pretrained(
    config["pretrained_teacher_model"], subfolder="scheduler"
)
tokenizer = AutoTokenizer.from_pretrained(
    config["pretrained_teacher_model"], subfolder="tokenizer", use_fast=False
)
text_encoder = CLIPTextModel.from_pretrained(
    config["pretrained_teacher_model"], subfolder="text_encoder"
)
vae = AutoencoderKL.from_pretrained(
    config["pretrained_teacher_model"], subfolder="vae"
)
teacher_unet = UNet2DConditionModel.from_pretrained(
    config["pretrained_teacher_model"], subfolder="unet"
)

# Freeze teacher models
vae.requires_grad_(False)
text_encoder.requires_grad_(False)
teacher_unet.requires_grad_(False)

# Create student U-Net
unet = UNet2DConditionModel.from_pretrained(
    config["pretrained_teacher_model"], subfolder="unet"
)
unet.train()

def extract_into_tensor(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

class DDIMSolver:
    def __init__(self, alpha_cumprods, timesteps=1000, ddim_timesteps=50):
        step_ratio = timesteps // ddim_timesteps
        self.ddim_timesteps = (np.arange(1, ddim_timesteps + 1) * step_ratio).round().astype(np.int64) - 1
        self.ddim_alpha_cumprods = alpha_cumprods[self.ddim_timesteps]
        self.ddim_alpha_cumprods_prev = np.asarray(
            [alpha_cumprods[0]] + alpha_cumprods[self.ddim_timesteps[:-1]].tolist()
        )
        self.ddim_timesteps = torch.from_numpy(self.ddim_timesteps).long()
        self.ddim_alpha_cumprods = torch.from_numpy(self.ddim_alpha_cumprods)
        self.ddim_alpha_cumprods_prev = torch.from_numpy(self.ddim_alpha_cumprods_prev)

    def to(self, device):
        self.ddim_timesteps = self.ddim_timesteps.to(device)
        self.ddim_alpha_cumprods = self.ddim_alpha_cumprods.to(device)
        self.ddim_alpha_cumprods_prev = self.ddim_alpha_cumprods_prev.to(device)
        return self

    def ddim_step(self, pred_x0, pred_noise, timestep_index):
        alpha_cumprod_prev = extract_into_tensor(self.ddim_alpha_cumprods_prev, timestep_index, pred_x0.shape)
        dir_xt = (1.0 - alpha_cumprod_prev).sqrt() * pred_noise
        x_prev = alpha_cumprod_prev.sqrt() * pred_x0 + dir_xt
        return x_prev
# Load noise schedules and initialize DDIM solver
alpha_schedule = torch.sqrt(noise_scheduler.alphas_cumprod)
sigma_schedule = torch.sqrt(1 - noise_scheduler.alphas_cumprod)
solver = DDIMSolver(
    noise_scheduler.alphas_cumprod.numpy(),
    timesteps=noise_scheduler.config.num_train_timesteps,
    ddim_timesteps=50,
)

/home/node/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
class SDText2ImageDataset:
    def __init__(
        self,
        train_shards_path_or_url: Union[str, List[str]],
        num_train_examples: int,
        per_gpu_batch_size: int,
        global_batch_size: int,
        num_workers: int,
        resolution: int = 512,
        interpolation_type: str = "bilinear",
        shuffle_buffer_size: int = 1000,
        pin_memory: bool = False,
        persistent_workers: bool = False,
    ):
        if not isinstance(train_shards_path_or_url, str):
            train_shards_path_or_url = [list(braceexpand(urls)) for urls in train_shards_path_or_url]
            train_shards_path_or_url = list(itertools.chain.from_iterable(train_shards_path_or_url))

        interpolation_mode = resolve_interpolation_mode(interpolation_type)

        def transform(example):
            image = example["image"]
            image = TF.resize(image, resolution, interpolation=interpolation_mode)

            c_top, c_left, _, _ = transforms.RandomCrop.get_params(image, output_size=(resolution, resolution))
            image = TF.crop(image, c_top, c_left, resolution, resolution)
            image = TF.to_tensor(image)
            image = TF.normalize(image, [0.5], [0.5])

            example["image"] = image
            return example

        processing_pipeline = [
            wds.decode("pil", handler=wds.ignore_and_continue),
            wds.rename(image="jpg;png;jpeg;webp", text="text;txt;caption", handler=wds.warn_and_continue),
            wds.map(filter_keys({"image", "text"})),
            wds.map(transform),
            wds.to_tuple("image", "text"),
        ]

        pipeline = [
            wds.ResampledShards(train_shards_path_or_url),
            tarfile_to_samples_nothrow,
            wds.shuffle(shuffle_buffer_size),
            *processing_pipeline,
            wds.batched(per_gpu_batch_size, partial=False, collation_fn=default_collate),
        ]

        num_worker_batches = math.ceil(num_train_examples / (global_batch_size * num_workers))
        num_batches = num_worker_batches * num_workers
        num_samples = num_batches * global_batch_size

        self._train_dataset = wds.DataPipeline(*pipeline).with_epoch(num_worker_batches)
        self._train_dataloader = wds.WebLoader(
            self._train_dataset,
            batch_size=None,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=persistent_workers,
        )
        self._train_dataloader.num_batches = num_batches
        self._train_dataloader.num_samples = num_samples

    @property
    def train_dataset(self):
        return self._train_dataset

    @property
    def train_dataloader(self):
        return self._train_dataloader

In [8]:
def log_validation(vae, unet, config, accelerator, weight_dtype, step):
    logger.info("Running validation...")
    if torch.backends.mps.is_available():
        autocast_ctx = nullcontext()
    else:
        autocast_ctx = torch.autocast(accelerator.device.type, dtype=weight_dtype)

    unet = accelerator.unwrap_model(unet)
    pipeline = StableDiffusionPipeline.from_pretrained(
        config["pretrained_teacher_model"],
        vae=vae,
        scheduler=LCMScheduler.from_pretrained(config["pretrained_teacher_model"], subfolder="scheduler"),
        revision=config["revision"],
        torch_dtype=weight_dtype,
        safety_checker=None,
    )
    pipeline.set_progress_bar_config(disable=True)

    lora_state_dict = get_module_kohya_state_dict(unet, "lora_unet", weight_dtype)
    pipeline.load_lora_weights(lora_state_dict)
    pipeline.fuse_lora()

    pipeline = pipeline.to(accelerator.device, dtype=weight_dtype)
    if config.get("enable_xformers_memory_efficient_attention", False):
        pipeline.enable_xformers_memory_efficient_attention()

    if config.get("seed", None) is None:
        generator = None
    else:
        generator = torch.Generator(device=accelerator.device).manual_seed(config["seed"])

    validation_prompts = [
        "portrait photo of a girl, photograph, highly detailed face, depth of field, moody light, golden hour, style by Dan Winters, Russell James, Steve McCurry, centered, extremely detailed, Nikon D850, award winning photography",
        "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k",
        "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k",
        "A photo of beautiful mountain with realistic sunset and blue lake, highly detailed, masterpiece",
    ]

    for _, prompt in enumerate(validation_prompts):
        with autocast_ctx:
            images = pipeline(
                prompt=prompt,
                num_inference_steps=4,
                num_images_per_prompt=4,
                generator=generator,
                guidance_scale=1.0,
            ).images

            for _, image in enumerate(images):
                image.save(f"{config['output_dir']}/val_image_{step}_{_}.png")

    del pipeline
    gc.collect()
    torch.cuda.empty_cache()

In [9]:
# From LatentConsistencyModel.get_guidance_scale_embedding
def guidance_scale_embedding(w, embedding_dim=512, dtype=torch.float32):
    assert len(w.shape) == 1
    w = w * 1000.0

    half_dim = embedding_dim // 2
    emb = torch.log(torch.tensor(10000.0)) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=dtype) * -emb)
    emb = w.to(dtype)[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # zero pad
        emb = torch.nn.functional.pad(emb, (0, 1))
    assert emb.shape == (w.shape[0], embedding_dim)
    return emb


def append_dims(x, target_dims):
    dims_to_append = target_dims - x.ndim
    if dims_to_append < 0:
        raise ValueError(f"input has {x.ndim} dims but target_dims is {target_dims}, which is less")
    return x[(...,) + (None,) * dims_to_append]


# From LCMScheduler.get_scalings_for_boundary_condition_discrete
def scalings_for_boundary_conditions(timestep, sigma_data=0.5, timestep_scaling=10.0):
    scaled_timestep = timestep_scaling * timestep
    c_skip = sigma_data**2 / (scaled_timestep**2 + sigma_data**2)
    c_out = scaled_timestep / (scaled_timestep**2 + sigma_data**2) ** 0.5
    return c_skip, c_out


# Compare LCMScheduler.step, Step 4
def get_predicted_original_sample(model_output, timesteps, sample, prediction_type, alphas, sigmas):
    alphas = extract_into_tensor(alphas, timesteps, sample.shape)
    sigmas = extract_into_tensor(sigmas, timesteps, sample.shape)
    if prediction_type == "epsilon":
        pred_x_0 = (sample - sigmas * model_output) / alphas
    elif prediction_type == "sample":
        pred_x_0 = model_output
    elif prediction_type == "v_prediction":
        pred_x_0 = alphas * sample - sigmas * model_output
    else:
        raise ValueError(
            f"Prediction type {prediction_type} is not supported; currently, `epsilon`, `sample`, and `v_prediction`"
            f" are supported."
        )

    return pred_x_0


# Based on step 4 in DDIMScheduler.step
def get_predicted_noise(model_output, timesteps, sample, prediction_type, alphas, sigmas):
    alphas = extract_into_tensor(alphas, timesteps, sample.shape)
    sigmas = extract_into_tensor(sigmas, timesteps, sample.shape)
    if prediction_type == "epsilon":
        pred_epsilon = model_output
    elif prediction_type == "sample":
        pred_epsilon = (sample - alphas * model_output) / sigmas
    elif prediction_type == "v_prediction":
        pred_epsilon = alphas * model_output + sigmas * sample
    else:
        raise ValueError(
            f"Prediction type {prediction_type} is not supported; currently, `epsilon`, `sample`, and `v_prediction`"
            f" are supported."
        )

    return pred_epsilon


def extract_into_tensor(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))


class DDIMSolver:
    def __init__(self, alpha_cumprods, timesteps=1000, ddim_timesteps=50):
        step_ratio = timesteps // ddim_timesteps
        self.ddim_timesteps = (np.arange(1, ddim_timesteps + 1) * step_ratio).round().astype(np.int64) - 1
        self.ddim_alpha_cumprods = alpha_cumprods[self.ddim_timesteps]
        self.ddim_alpha_cumprods_prev = np.asarray(
            [alpha_cumprods[0]] + alpha_cumprods[self.ddim_timesteps[:-1]].tolist()
        )
        self.ddim_timesteps = torch.from_numpy(self.ddim_timesteps).long()
        self.ddim_alpha_cumprods = torch.from_numpy(self.ddim_alpha_cumprods)
        self.ddim_alpha_cumprods_prev = torch.from_numpy(self.ddim_alpha_cumprods_prev)

    def to(self, device):
        self.ddim_timesteps = self.ddim_timesteps.to(device)
        self.ddim_alpha_cumprods = self.ddim_alpha_cumprods.to(device)
        self.ddim_alpha_cumprods_prev = self.ddim_alpha_cumprods_prev.to(device)
        return self

    def ddim_step(self, pred_x0, pred_noise, timestep_index):
        alpha_cumprod_prev = extract_into_tensor(self.ddim_alpha_cumprods_prev, timestep_index, pred_x0.shape)
        dir_xt = (1.0 - alpha_cumprod_prev).sqrt() * pred_noise
        x_prev = alpha_cumprod_prev.sqrt() * pred_x0 + dir_xt
        return x_prev


@torch.no_grad()
def update_ema(target_params, source_params, rate=0.99):
    for targ, src in zip(target_params, source_params):
        targ.detach().mul_(rate).add_(src, alpha=1 - rate)

In [10]:
def import_model_class_from_model_name_or_path(
    pretrained_model_name_or_path: str, revision: str, subfolder: str = "text_encoder"
):
    text_encoder_config = PretrainedConfig.from_pretrained(
        pretrained_model_name_or_path, subfolder=subfolder, revision=revision
    )
    model_class = text_encoder_config.architectures[0]

    if model_class == "CLIPTextModel":
        from transformers import CLIPTextModel

        return CLIPTextModel
    elif model_class == "CLIPTextModelWithProjection":
        from transformers import CLIPTextModelWithProjection

        return CLIPTextModelWithProjection
    else:
        raise ValueError(f"{model_class} is not supported.")


# Adapted from pipelines.StableDiffusionPipeline.encode_prompt
def encode_prompt(prompt_batch, text_encoder, tokenizer, proportion_empty_prompts, is_train=True):
    captions = []
    for caption in prompt_batch:
        if random.random() < proportion_empty_prompts:
            captions.append("")
        elif isinstance(caption, str):
            captions.append(caption)
        elif isinstance(caption, (list, np.ndarray)):
            captions.append(random.choice(caption) if is_train else caption[0])

    with torch.no_grad():
        text_inputs = tokenizer(
            captions,
            padding="max_length",
            max_length=tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        text_input_ids = text_inputs.input_ids
        prompt_embeds = text_encoder(text_input_ids.to(text_encoder.device))[0]

    return prompt_embeds

In [11]:

# Set up accelerator
accelerator_project_config = ProjectConfiguration(project_dir=config["output_dir"], logging_dir=os.path.join(config["output_dir"], "logs"))
accelerator = Accelerator(
    gradient_accumulation_steps=config["gradient_accumulation_steps"],
    mixed_precision=config["mixed_precision"],
    log_with="tensorboard",
    project_config=accelerator_project_config,
)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    transformers.utils.logging.set_verbosity_warning()
    diffusers.utils.logging.set_verbosity_info()
else:
    transformers.utils.logging.set_verbosity_error()
    diffusers.utils.logging.set_verbosity_error()

# Set random seed
if config["seed"] is not None:
    set_seed(config["seed"])

# Create output directory
os.makedirs(config["output_dir"], exist_ok=True)

# Load models and create solver
noise_scheduler = DDPMScheduler.from_pretrained(config["pretrained_teacher_model"], subfolder="scheduler")
alpha_schedule = torch.sqrt(noise_scheduler.alphas_cumprod)
sigma_schedule = torch.sqrt(1 - noise_scheduler.alphas_cumprod)
solver = DDIMSolver(noise_scheduler.alphas_cumprod.numpy(), timesteps=noise_scheduler.config.num_train_timesteps, ddim_timesteps=50)

tokenizer = AutoTokenizer.from_pretrained(config["pretrained_teacher_model"], subfolder="tokenizer", use_fast=False)
text_encoder = CLIPTextModel.from_pretrained(config["pretrained_teacher_model"], subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(config["pretrained_teacher_model"], subfolder="vae")
teacher_unet = UNet2DConditionModel.from_pretrained(config["pretrained_teacher_model"], subfolder="unet")

vae.requires_grad_(False)
text_encoder.requires_grad_(False)
teacher_unet.requires_grad_(False)

unet = UNet2DConditionModel.from_pretrained(config["pretrained_teacher_model"], subfolder="unet")
unet.train()

if accelerator.unwrap_model(unet).dtype != torch.float32:
    raise ValueError(f"Student UNet loaded as datatype {accelerator.unwrap_model(unet).dtype}. Please make sure to load the model weights in full precision.")

# Add LoRA to the student UNet
lora_config = LoraConfig(r=config["lora_rank"], target_modules=["to_q", "to_k", "to_v", "to_out.0", "proj_in", "proj_out", "ff.net.0.proj", "ff.net.2", "conv1", "conv2", "conv_shortcut", "downsamplers.0.conv", "upsamplers.0.conv", "time_emb_proj"], lora_alpha=config["lora_rank"], lora_dropout=0.0)
unet = get_peft_model(unet, lora_config)

# Handle mixed precision and device placement
weight_dtype = torch.float32
if accelerator.mixed_precision == "fp16":
    weight_dtype = torch.float16
elif accelerator.mixed_precision == "bf16":
    weight_dtype = torch.bfloat16

vae.to(accelerator.device)
text_encoder.to(accelerator.device, dtype=weight_dtype)
teacher_unet.to(accelerator.device)

alpha_schedule = alpha_schedule.to(accelerator.device)
sigma_schedule = sigma_schedule.to(accelerator.device)
solver = solver.to(accelerator.device)

# Create optimizer
if config["use_8bit_adam"]:
    try:
        import bitsandbytes as bnb
        optimizer_class = bnb.optim.AdamW8bit
    except ImportError:
        raise ImportError("To use 8-bit Adam, please install the bitsandbytes library: `pip install bitsandbytes`.")
else:
    optimizer_class = torch.optim.AdamW

optimizer = optimizer_class(
    unet.parameters(),
    lr=config["learning_rate"],
    betas=(0.9, 0.999),
    weight_decay=config["adam_weight_decay"],
    eps=1e-08,
)
def compute_embeddings(prompt_batch, proportion_empty_prompts, text_encoder, tokenizer, is_train=True):
    prompt_embeds = encode_prompt(prompt_batch, text_encoder, tokenizer, proportion_empty_prompts, is_train)
    return {"prompt_embeds": prompt_embeds}
# Create dataset and dataloader
dataset = SDText2ImageDataset(
    train_shards_path_or_url=config["train_shards_path_or_url"],
    num_train_examples=config["max_train_samples"],
    per_gpu_batch_size=config["train_batch_size"],
    global_batch_size=config["train_batch_size"] * accelerator.num_processes,
    num_workers=config["dataloader_num_workers"],
    resolution=config["resolution"],
    interpolation_type="bilinear",
    shuffle_buffer_size=1000,
    pin_memory=True,
    persistent_workers=True,
)
train_dataloader = dataset.train_dataloader

compute_embeddings_fn = functools.partial(
compute_embeddings,
proportion_empty_prompts=0,
text_encoder=text_encoder,
tokenizer=tokenizer,
)

# Set up learning rate scheduler
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(train_dataloader.num_batches / config["gradient_accumulation_steps"])
if config["max_train_steps"] is None:
    config["max_train_steps"] = config["num_train_epochs"] * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    config["lr_scheduler"],
    optimizer=optimizer,
    num_warmup_steps=config.get("lr_warmup_steps", 0),
    num_training_steps=config["max_train_steps"],
)

# Prepare everything with accelerator
unet, optimizer, lr_scheduler = accelerator.prepare(unet, optimizer, lr_scheduler)

# Recalculate total training steps
num_update_steps_per_epoch = math.ceil(train_dataloader.num_batches / config["gradient_accumulation_steps"])
if overrode_max_train_steps:
    config["max_train_steps"] = config["num_train_epochs"] * num_update_steps_per_epoch
config["num_train_epochs"] = math.ceil(config["max_train_steps"] / num_update_steps_per_epoch)

# Initialize trackers
if accelerator.is_main_process:
    accelerator.init_trackers("lcm-lora", config=config)

uncond_input_ids = tokenizer([""] * config["train_batch_size"], return_tensors="pt", padding="max_length", max_length=77).input_ids.to(accelerator.device)
uncond_prompt_embeds = text_encoder(uncond_input_ids)[0]

if torch.backends.mps.is_available():
    autocast_ctx = nullcontext()
else:
    autocast_ctx = torch.autocast(accelerator.device.type)

# Train!
total_batch_size = config["train_batch_size"] * accelerator.num_processes * config["gradient_accumulation_steps"]
logger.info("***** Running training *****")
logger.info(f"  Num examples = {train_dataloader.num_samples}")
logger.info(f"  Num Epochs = {config['num_train_epochs']}")
logger.info(f"  Instantaneous batch size per device = {config['train_batch_size']}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {config['gradient_accumulation_steps']}")
logger.info(f"  Total optimization steps = {config['max_train_steps']}")
global_step = 0
first_epoch = 0

progress_bar = tqdm(range(global_step, config["max_train_steps"]), disable=not accelerator.is_local_main_process)

for epoch in range(first_epoch, config["num_train_epochs"]):
    for step, batch in enumerate(train_dataloader):
        with accelerator.accumulate(unet):
            # Load and process the image and text conditioning
            image, text = batch
            image = image.to(accelerator.device, non_blocking=True)
            encoded_text = compute_embeddings_fn(text)

            pixel_values = image.to(dtype=weight_dtype)
            if vae.dtype != weight_dtype:
                vae.to(dtype=weight_dtype)

            # Encode pixel values with batch size of at most config["vae_encode_batch_size"]
            latents = []
            for i in range(0, pixel_values.shape[0], config["vae_encode_batch_size"]):
                latents.append(vae.encode(pixel_values[i : i + config["vae_encode_batch_size"]]).latent_dist.sample())
            latents = torch.cat(latents, dim=0)

            latents = latents * vae.config.scaling_factor
            latents = latents.to(weight_dtype)
            bsz = latents.shape[0]

            # Sample random timestep and starting timestep for each image
            topk = noise_scheduler.config.num_train_timesteps // 50
            index = torch.randint(0, 50, (bsz,), device=latents.device).long()
            start_timesteps = solver.ddim_timesteps[index]
            timesteps = start_timesteps - topk
            timesteps = torch.where(timesteps < 0, torch.zeros_like(timesteps), timesteps)

            # Get boundary scalings for start and end timesteps
            c_skip_start, c_out_start = scalings_for_boundary_conditions(start_timesteps, timestep_scaling=10.0)
            c_skip_start, c_out_start = [append_dims(x, latents.ndim) for x in [c_skip_start, c_out_start]]
            c_skip, c_out = scalings_for_boundary_conditions(timesteps, timestep_scaling=10.0)
            c_skip, c_out = [append_dims(x, latents.ndim) for x in [c_skip, c_out]]

            # Sample noise and add it to the latents
            noise = torch.randn_like(latents)
            noisy_model_input = noise_scheduler.add_noise(latents, noise, start_timesteps)

            # Sample random guidance scale
            w = (config["w_max"] - config["w_min"]) * torch.rand((bsz,)) + config["w_min"]
            w = w.reshape(bsz, 1, 1, 1).to(latents.device, dtype=latents.dtype)

            prompt_embeds = encoded_text["prompt_embeds"]
            noise_pred = unet(noisy_model_input, start_timesteps, timestep_cond=None, encoder_hidden_states=prompt_embeds.float(), added_cond_kwargs=encoded_text).sample

            pred_x_0 = get_predicted_original_sample(noise_pred, start_timesteps, noisy_model_input, noise_scheduler.config.prediction_type, alpha_schedule, sigma_schedule)
            model_pred = c_skip_start * noisy_model_input + c_out_start * pred_x_0

            with torch.no_grad():
                with autocast_ctx:
                    # Get teacher model predictions
                    cond_teacher_output = teacher_unet(noisy_model_input.to(weight_dtype), start_timesteps, encoder_hidden_states=prompt_embeds.to(weight_dtype)).sample
                    cond_pred_x0 = get_predicted_original_sample(cond_teacher_output, start_timesteps, noisy_model_input, noise_scheduler.config.prediction_type, alpha_schedule, sigma_schedule)
                    cond_pred_noise = get_predicted_noise(cond_teacher_output, start_timesteps, noisy_model_input, noise_scheduler.config.prediction_type, alpha_schedule, sigma_schedule)

                    uncond_teacher_output = teacher_unet(noisy_model_input.to(weight_dtype), start_timesteps, encoder_hidden_states=uncond_prompt_embeds.to(weight_dtype)).sample
                    uncond_pred_x0 = get_predicted_original_sample(uncond_teacher_output, start_timesteps, noisy_model_input, noise_scheduler.config.prediction_type, alpha_schedule, sigma_schedule)
                    uncond_pred_noise = get_predicted_noise(uncond_teacher_output, start_timesteps, noisy_model_input, noise_scheduler.config.prediction_type, alpha_schedule, sigma_schedule)

                    # Calculate CFG estimates
                    pred_x0 = cond_pred_x0 + w * (cond_pred_x0 - uncond_pred_x0)
                    pred_noise = cond_pred_noise + w * (cond_pred_noise - uncond_pred_noise)

                    # Run one step of ODE solver
                    x_prev = solver.ddim_step(pred_x0, pred_noise, index)

            # Get target LCM prediction
            with torch.no_grad():
                with autocast_ctx:
                    target_noise_pred = unet(x_prev.float(), timesteps, timestep_cond=None, encoder_hidden_states=prompt_embeds.float()).sample
                pred_x_0 = get_predicted_original_sample(target_noise_pred, timesteps, x_prev, noise_scheduler.config.prediction_type, alpha_schedule, sigma_schedule)
                target = c_skip * x_prev + c_out * pred_x_0

            # Calculate loss
            if config["loss_type"] == "l2":
                loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
            elif config["loss_type"] == "huber":
                loss = torch.mean(torch.sqrt((model_pred.float() - target.float()) ** 2 + 0.001**2) - 0.001)

            # Backpropagate and update weights
            accelerator.backward(loss)
            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(unet.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            progress_bar.update(1)
            global_step += 1

            # Log training progress
            if accelerator.is_main_process:
                logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
                progress_bar.set_postfix(**logs)
                accelerator.log(logs, step=global_step)

                # Save model checkpoints
                if global_step % config["checkpointing_steps"] == 0:
                    save_path = os.path.join(config["output_dir"], f"checkpoint-{global_step}")
                    accelerator.save_state(save_path)
                    logger.info(f"Saved state to {save_path}")

                # Run validation
                if global_step % config["validation_steps"] == 0:
                    log_validation(vae, unet, config, accelerator, weight_dtype, global_step)

        if global_step >= config["max_train_steps"]:
            break

# Save the final model
accelerator.wait_for_everyone()
if accelerator.is_main_process:
    unet = accelerator.unwrap_model(unet)
    unet.save_pretrained(config["output_dir"])
    lora_state_dict = get_peft_model_state_dict(unet, adapter_name="default")
    StableDiffusionPipeline.save_lora_weights(os.path.join(config["output_dir"], "unet_lora"), lora_state_dict)

accelerator.end_training()



06/09/2024 18:58:39 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'rescale_betas_zero_snr', 'clip_sample_range', 'dynamic_thresholding_ratio', 'variance_type', 'timestep_spacing', 'prediction_type', 'thresholding', 'sample_max_value'} was not found in config. Values will be initialized to default values.
/home/node/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
{'latents_mean', 'scaling_factor', 'force_upcast', 'latents_std'} was not found in config. Values will be initialized to default values.
{'num_class_embeds', 'class_embeddings_concat', 'mid_block_type', 'attention_type', 'conv_out_kernel', 'resnet_time_scale_shift', 'only_cross_attention', 'time_embe

: 